# **DESARROLLO DE FUNCIONES DE CONSULTA**
Aquí se muestran las funciones que responden a los planteamientos del proyecto individual 1, este procedimiento consistió en el procesamiento final de los datos hasta transformarlos en información lista para ser consumida, se unieron tablas y se eliminaron datos innecesarios para los cálculos, se generarán entonces 5 archivos que contienen la data que será consultada directamente desde la API, así mismo se dearrollan las funciones finales para dichas consultas.

In [1]:
import pandas as pd
from IPython.display import display
from ast import literal_eval

Para el desarrollo de las funciones es imprescindible cargar los archivos generados en la etapa de ETL.

In [2]:
output=pd.read_csv('output90p.csv')
reviews=pd.read_csv('reviews90p.csv')
items=pd.read_csv('items.csv')

## 1. FUNCIÓN def PlayTimeGenre
def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

1.1. Generamos un dataframe agrupando las columnas que nos interesan item id y playtime_forever en una columna llamada total playtime, el nombre de este df sera items2

In [3]:
gdata = items.groupby('item_id')['playtime_forever'].sum().reset_index()

items2 = pd.DataFrame({
    'item_id': gdata['item_id'],
    'total_playtime': gdata['playtime_forever']
})
items2

,item_id,total_playtime
0,10,17107858
1,20,960524
2,30,756375
3,40,154424
4,50,726545
...,...,...
10973,528580,0
10974,528660,182
10975,529670,0
10976,529820,0


1.2. Generamos otro data frame con las columnas de interes de la tabla output

In [4]:
output2=output[['id','year','genres']]
output2

,id,year,genres
0,761140,2018,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,2018,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,2017,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,2017,"['Action', 'Adventure', 'Casual']"
4,773570,2014,"['Action', 'Casual', 'Indie', 'Sports']"
...,...,...,...
32078,773640,2018,"['Casual', 'Indie', 'Simulation', 'Strategy']"
32079,733530,2018,"['Casual', 'Indie', 'Strategy']"
32080,610660,2018,"['Indie', 'Racing', 'Simulation']"
32081,658870,2017,"['Casual', 'Indie']"


1.3. Los unimos considerando el id como columna comun

In [5]:
playtimegenre = pd.merge(output2, items2, left_on='id', right_on='item_id')

In [6]:
playtimegenre

,id,year,genres,item_id,total_playtime
0,282010,1997,"['Action', 'Indie', 'Racing']",282010,9319
1,70,1998,['Action'],70,2650946
2,1640,2006,['Strategy'],1640,27162
3,1630,2006,['Strategy'],1630,21105
4,2400,2006,"['Action', 'Indie', 'RPG']",2400,1030947
...,...,...,...,...,...
9173,259340,2003,"['Action', 'Casual', 'Indie']",259340,371
9174,30,2003,['Action'],30,756375
9175,200980,2003,"['Strategy', 'RPG', 'Indie']",200980,3393
9176,13230,2004,['Action'],13230,345093


1.4. Agrupamos las listas de generos repetidas sumando el tiempo de juego

In [7]:
playtimegenre2=playtimegenre.groupby(['genres', 'year'])['total_playtime'].sum().reset_index()

In [8]:
playtimegenre2

,genres,year,total_playtime
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",2015,37249
1,"['Action', 'Adventure', 'Casual', 'Free to Pla...",2015,37824
2,"['Action', 'Adventure', 'Casual', 'Free to Pla...",2014,0
3,"['Action', 'Adventure', 'Casual', 'Free to Pla...",2014,366794
4,"['Action', 'Adventure', 'Casual', 'Free to Pla...",2016,3766
...,...,...,...
2009,[],2012,727
2010,[],2013,1
2011,[],2014,453
2012,[],2015,16492


1.5. Desempaquetamos los generos 

In [10]:
# Convertimos la cadena en una lista real
playtimegenre2['genres'] = playtimegenre2['genres'].apply(literal_eval)

# Desempacamos la lista en varias filas
playtimegenre3 = playtimegenre2.explode('genres')

In [11]:
playtimegenre3

,genres,year,total_playtime
0,Action,2015,37249
0,Adventure,2015,37249
0,Casual,2015,37249
0,Free to Play,2015,37249
0,Indie,2015,37249
...,...,...,...
2009,NaN,2012,727
2010,NaN,2013,1
2011,NaN,2014,453
2012,NaN,2015,16492


1.6. Agrupamos los géneros sumando el tiempo de juego respetando la columna year

In [12]:
playtimegenre4 = playtimegenre3.groupby(['genres', 'year'])['total_playtime'].sum().reset_index()

In [13]:
playtimegenre4

,genres,year,total_playtime
0,Action,1990,18335
1,Action,1991,2502
2,Action,1992,4988
3,Action,1993,24623
4,Action,1994,120177
...,...,...,...
335,Web Publishing,2013,333678
336,Web Publishing,2014,33641
337,Web Publishing,2015,348673
338,Web Publishing,2016,136


In [14]:
playtimegenre4.to_parquet('playtimegenre4.parquet')

In [15]:
playtimegenre4=pd.read_parquet('playtimegenre4.parquet')

1.7. Creamos la funcion, vamos a tener en cuenta las minusculas y mayusculas para hacerlo más amigable y reducir la posibilidad de errores

In [16]:
def PlayTimeGenre(genero):
    # Convertir el género ingresado a minúsculas
    genero_lower = genero.lower()

    # Convertir los géneros en el conjunto de datos a minúsculas y verificar la pertenencia
    if genero_lower not in playtimegenre4['genres'].str.lower().unique():
        return {"Género no pertenece al conjunto de datos"}

    # Filtrar el DataFrame para el género especificado
    genre_data = playtimegenre4[playtimegenre4['genres'].str.lower() == genero_lower]

    # Encontrar el año con el mayor tiempo total de juego
    max_year = genre_data.loc[genre_data['total_playtime'].idxmax(), 'year']
    
    out = {"Año de lanzamiento con más horas jugadas para {}:".format(genero): max_year}
    return out

In [21]:
PlayTimeGenre('web publishing')

{'Año de lanzamiento con más horas jugadas para web publishing:': 2012}

## 2. FUNCIÓN def UserForGenre
def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

2.1. Creamos  dataframes con las columnas de interés

In [22]:
dfitems2=items[['user_id','item_id','playtime_forever']]
dfitems2

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0
...,...,...,...
5094100,76561198329548331,346330,0
5094101,76561198329548331,373330,0
5094102,76561198329548331,388490,3
5094103,76561198329548331,521570,4


In [23]:
#usamos output2
output2.head()

,id,year,genres
0,761140,2018,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,2018,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,2017,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,2017,"['Action', 'Adventure', 'Casual']"
4,773570,2014,"['Action', 'Casual', 'Indie', 'Sports']"


2.2. Unimos los dataframes considerando id como columna común

In [24]:
userforgenre = pd.merge(output2, dfitems2, left_on='id', right_on='item_id')

In [25]:
userforgenre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4242849 entries, 0 to 4242848
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   id                int64 
 1   year              int64 
 2   genres            object
 3   user_id           object
 4   item_id           int64 
 5   playtime_forever  int64 
dtypes: int64(4), object(2)
memory usage: 226.6+ MB


2.3. Desempacamos la lista de generos ignoramos valores Nan de modo similar al del problema 1

In [26]:
userforgenre['genres'] = userforgenre['genres'].apply(lambda x: literal_eval(x) if pd.notna(x) else x)
userforgenre2 = userforgenre.explode('genres')

In [27]:
userforgenre2

,id,year,genres,user_id,item_id,playtime_forever
0,282010,1997,Action,UTNerd24,282010,5
0,282010,1997,Indie,UTNerd24,282010,5
0,282010,1997,Racing,UTNerd24,282010,5
1,282010,1997,Action,I_DID_911_JUST_SAYING,282010,0
1,282010,1997,Indie,I_DID_911_JUST_SAYING,282010,0
...,...,...,...,...,...,...
4242844,80,2004,Action,76561198273508956,80,0
4242845,80,2004,Action,76561198282090798,80,0
4242846,80,2004,Action,943525,80,0
4242847,80,2004,Action,76561198283312749,80,9


2.4. Reagrupamos considerando que user_id puede jugar diversos juegos con el mismo genero y para efectos de la pregunta nos interesa el tiempo total en el genero, de modo que cada user queda con un genero por año (year)

In [28]:
userforgenre3 = userforgenre2.groupby(['user_id', 'year', 'genres'], as_index=False)['playtime_forever'].sum()

In [29]:
userforgenre3

,user_id,year,genres,playtime_forever
0,--000--,2006,Indie,925
1,--000--,2006,Simulation,925
2,--000--,2009,Action,5329
3,--000--,2010,Action,22
4,--000--,2010,Free to Play,22
...,...,...,...,...
3518299,zzzmidmiss,2016,Casual,0
3518300,zzzmidmiss,2016,Early Access,0
3518301,zzzmidmiss,2016,Indie,0
3518302,zzzmidmiss,2016,Simulation,0


2.5. Agrupamos solo por género y usuario  para posteriormente sacar los usuarios con más tiempo por genero. El proposito es obtener los usuarios con más tiempo por género en total

In [30]:

userforgenre4 = userforgenre3.groupby(['user_id', 'genres'], as_index=False)['playtime_forever'].sum()
userforgenre4

,user_id,genres,playtime_forever
0,--000--,Action,139885
1,--000--,Adventure,12138
2,--000--,Casual,16135
3,--000--,Early Access,531
4,--000--,Free to Play,20448
...,...,...,...
677994,zzzmidmiss,RPG,2337
677995,zzzmidmiss,Racing,9
677996,zzzmidmiss,Simulation,16
677997,zzzmidmiss,Sports,210


2.6. Seleccionamos los generos unicos con mayor tiempo, mostrando tambien el user_id

In [31]:
max_playtime_rows1 = userforgenre4.loc[userforgenre4.groupby('genres')['playtime_forever'].idxmax()]


In [32]:
max_playtime_rows1

,user_id,genres,playtime_forever
491286,Sp3ctre,Action,1782253
477853,REBAS_AS_F-T,Adventure,2195886
485117,ScottyG555,Animation &amp; Modeling,168314
453286,Lickidactyl,Audio Production,109916
477856,REBAS_AS_F-T,Casual,1231384
485119,ScottyG555,Design &amp; Illustration,168314
18307,76561197978756659,Early Access,316969
526599,ballig,Education,103651
577274,idonothack,Free to Play,808242
477861,REBAS_AS_F-T,Indie,2412793


2.7. Unimos este último dataframe con el dataframe userforgenre3 con el proposito de solo quedarnos con los usuarios que tienen más tiempo jugado pero también poder visualizar años y tiempo jugado por año

In [33]:
# Fusionar data sets
merged_df = pd.merge(userforgenre3, max_playtime_rows1[['user_id', 'genres']], on=['user_id', 'genres'], how='inner')

# Seleccion de columnas de interes
merged_df = merged_df[['user_id', 'genres', 'playtime_forever', 'year']]

In [34]:
merged_df

,user_id,genres,playtime_forever,year
0,76561197978756659,Early Access,316786,2013
1,76561197978756659,Early Access,183,2015
2,76561197995725436,Sports,40941,2012
3,76561197995725436,Sports,79186,2013
4,76561197995725436,Sports,73079,2014
...,...,...,...,...
168,shinomegami,RPG,107775,2013
169,shinomegami,RPG,81939,2014
170,shinomegami,RPG,111788,2015
171,shinomegami,RPG,15550,2016


2.8. Calculamos columna horas

In [35]:
merged_df['playtime_forever_h'] = merged_df['playtime_forever'] / 60
merged_df['playtime_forever_h']=merged_df['playtime_forever_h'].round(2)
merged_df

,user_id,genres,playtime_forever,year,playtime_forever_h
0,76561197978756659,Early Access,316786,2013,5279.77
1,76561197978756659,Early Access,183,2015,3.05
2,76561197995725436,Sports,40941,2012,682.35
3,76561197995725436,Sports,79186,2013,1319.77
4,76561197995725436,Sports,73079,2014,1217.98
...,...,...,...,...,...
168,shinomegami,RPG,107775,2013,1796.25
169,shinomegami,RPG,81939,2014,1365.65
170,shinomegami,RPG,111788,2015,1863.13
171,shinomegami,RPG,15550,2016,259.17


In [36]:
merged_df.to_parquet('usuariostime.parquet')

In [37]:
merged_df=pd.read_parquet('usuariostime.parquet')

2.9. Creamos la función

In [38]:
def UserForGenre(genero):
    if genero not in merged_df['genres'].unique():
        return {"Género no pertenece al conjunto de datos"}
    
    grouped_df = merged_df[merged_df['genres'] == genero].groupby(['user_id', 'year']).agg({'playtime_forever_h': 'sum'}).reset_index()
    playtime_list = []
    user_ids = grouped_df['user_id'].tolist()
    user_id1 = str(user_ids[0])
    
    for index, row in grouped_df.iterrows():
        playtime_dict = {
            'Año': row['year'],
            'Horas': row['playtime_forever_h']
        }
        playtime_list.append(playtime_dict)
    
    
    out={"Usuario con más horas jugadas para {}:" .format(genero) : user_id1 , "Horas jugadas":playtime_list}
    
    return out

In [39]:
UserForGenre('Action')

{'Usuario con más horas jugadas para Action:': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1993, 'Horas': 0.0},
  {'Año': 1995, 'Horas': 3.62},
  {'Año': 1996, 'Horas': 0.0},
  {'Año': 1998, 'Horas': 0.0},
  {'Año': 1999, 'Horas': 0.73},
  {'Año': 2000, 'Horas': 1177.4},
  {'Año': 2001, 'Horas': 0.22},
  {'Año': 2002, 'Horas': 3.97},
  {'Año': 2003, 'Horas': 127.88},
  {'Año': 2004, 'Horas': 2123.52},
  {'Año': 2005, 'Horas': 355.65},
  {'Año': 2006, 'Horas': 14.93},
  {'Año': 2007, 'Horas': 1879.73},
  {'Año': 2008, 'Horas': 3.73},
  {'Año': 2009, 'Horas': 1805.43},
  {'Año': 2010, 'Horas': 1301.38},
  {'Año': 2011, 'Horas': 2581.6},
  {'Año': 2012, 'Horas': 6304.93},
  {'Año': 2013, 'Horas': 2007.68},
  {'Año': 2014, 'Horas': 2178.18},
  {'Año': 2015, 'Horas': 5215.07},
  {'Año': 2016, 'Horas': 531.55},
  {'Año': 2017, 'Horas': 2087.0}]}

## 3. FUNCION def UsersRecommend
def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Para esta función utilizaremos los data frames items y reviews

In [40]:
reviews.head()

,Unnamed: 0,posted,item_id,helpful,recommend,review,user_id,user_url,year,language,sent_an
0,0,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2
1,1,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2
2,2,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2
3,3,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,2014,en,2
4,4,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,2013,en,2


In [41]:
items

,Unnamed: 0,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...,...,...,...,...
5094100,5094100,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,346330,BrainBread 2,0,0
5094101,5094101,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0,0
5094102,5094102,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3,3
5094103,5094103,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4,4


3.1. Colocamos una columna con la recomendación en formato binario

In [42]:
reviews['binary_recommend'] = reviews['recommend'].apply(lambda x: int(x))

3.2. Creamos 2 dataframes extrayendo  las columnas que vamos a usar de reviews e items 

In [43]:
reviews1=reviews[['item_id', 'year', 'sent_an', 'binary_recommend']]
items1=items[['item_id', 'item_name']]

3.3 Reducimos datasets, para el dataset reviews1, reduciremos en funcion de item_id y year, consideraremos la suma de puntaje en sent_an y binary_recommend

In [44]:

reviews2 = reviews1.groupby(['item_id', 'year'], as_index=False)['sent_an', 'binary_recommend'].sum()
reviews2

C:\Users\Owner\AppData\Local\Temp\ipykernel_2944\2319182980.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  reviews2 = reviews1.groupby(['item_id', 'year'], as_index=False)['sent_an', 'binary_recommend'].sum()


,item_id,year,sent_an,binary_recommend
0,10,2011,2,1
1,10,2012,6,3
2,10,2013,9,8
3,10,2014,40,27
4,10,2015,31,16
...,...,...,...,...
6759,521430,2014,2,1
6760,521570,2013,0,1
6761,521570,2015,0,1
6762,521990,2013,2,1


3.4. Reducimos el dataset Items1 debido a que solo necesitamos valores unicos para item_name y de item_id

In [45]:
items2 = items1.drop_duplicates(subset=['item_id', 'item_name'])
items2

,item_id,item_name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force
...,...,...
4854174,375450,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden
5033865,353390,Alienware Steam Machine
5045944,354280,ChaosTower
5060350,433920,Aveyond 4: Shadow Of The Mist


3.5. Unimos los datasets con el proposito de trabajar unicamente con los items que tienen calificación y su correspondiente nombre de juego

In [46]:
recommend = pd.merge(reviews2, items2, left_on='item_id', right_on='item_id')
recommend

,item_id,year,sent_an,binary_recommend,item_name
0,10,2011,2,1,Counter-Strike
1,10,2012,6,3,Counter-Strike
2,10,2013,9,8,Counter-Strike
3,10,2014,40,27,Counter-Strike
4,10,2015,31,16,Counter-Strike
...,...,...,...,...,...
6249,521430,2014,2,1,Super Switch
6250,521570,2013,0,1,You Have 10 Seconds 2
6251,521570,2015,0,1,You Have 10 Seconds 2
6252,521990,2013,2,1,Galactic Storm


In [47]:
recommend.sort_values(by='binary_recommend') 

,item_id,year,sent_an,binary_recommend,item_name
5307,367710,2015,2,0,Afterlife Empire
555,13560,2014,1,0,Tom Clancy's Splinter Cell
556,13560,2015,2,0,Tom Clancy's Splinter Cell
3293,251670,2015,4,0,Battle Nations
1941,206480,2015,0,0,Dungeons & Dragons Online®
...,...,...,...,...,...
2293,218620,2014,735,464,PAYDAY 2
4375,304930,2014,940,604,Unturned
219,4000,2014,1313,824,Garry's Mod
95,730,2014,2188,1378,Counter-Strike: Global Offensive


3.6 Podemos reducir aun más, debido a que solo nos interesan los 3 juegos con mayor calificacion por year para la agrupación usamos el sentiment analysis y binary recommend.

In [48]:
top_items_per_year1 = recommend.groupby('year').apply(lambda x: x.nlargest(3, ['binary_recommend', 'sent_an'])).reset_index(drop=True)
top_items_per_year1

,item_id,year,sent_an,binary_recommend,item_name
0,1250,2010,9,6,Killing Floor
1,4000,2010,7,4,Garry's Mod
2,400,2010,4,3,Portal
3,620,2011,48,26,Portal 2
4,105600,2011,38,25,Terraria
5,4000,2011,36,21,Garry's Mod
6,105600,2012,72,46,Terraria
7,4000,2012,70,46,Garry's Mod
8,550,2012,53,37,Left 4 Dead 2
9,4000,2013,577,353,Garry's Mod


In [49]:
top_items_per_year1.to_parquet('topitemsperyear1.parquet')

In [50]:
top_items_per_year1=pd.read_parquet('topitemsperyear1.parquet')

3.7. Creamos la funcion de consulta



In [51]:
def UsersRecommend(year):
    if year not in top_items_per_year1['year'].unique():
        return {"No se tiene información respecto al año ingresado"}
    # Filter the DataFrame based on the given year
    selected_year = top_items_per_year1[top_items_per_year1['year'] == year]

    # Sort the values based on the 'binary_recommend' column in descending order
    sorted_items = selected_year.sort_values(by='binary_recommend', ascending=False)

    # Create a list of dictionaries with the desired format
    result_list = [{"Puesto {}: ".format(i+1): item_name} for i, item_name in enumerate(sorted_items['item_name'])]

    return result_list


In [52]:
UsersRecommend(2014)

[{'Puesto 1: ': 'Counter-Strike: Global Offensive'},
 {'Puesto 2: ': "Garry's Mod"},
 {'Puesto 3: ': 'Unturned'}]

## 4. FUNCION def UsersWorstDeveloper 
def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Utilizaremos los dataframes reviews y output

In [53]:
reviews.head()

,Unnamed: 0,posted,item_id,helpful,recommend,review,user_id,user_url,year,language,sent_an,binary_recommend
0,0,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2,1
1,1,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2,1
2,2,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2,1
3,3,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,2014,en,2,1
4,4,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,2013,en,2,1


In [54]:
output.head()

,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
0,0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140,Kotoshiro,2018
1,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980,Secret Level SRL,2018
2,2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,0.0,670290,Poolians.com,2017
3,3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,0.0,767400,彼岸领域,2017
4,4,NaN,"['Action', 'Casual', 'Indie', 'Sports']",Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,0.0,773570,NaN,2014


4.1. Tomamos las columnas necesarias para resolver el problema creando nuevos data frames.

In [55]:
dfoutput=output[['id','developer']]
dfoutput

,id,developer
0,761140,Kotoshiro
1,643980,Secret Level SRL
2,670290,Poolians.com
3,767400,彼岸领域
4,773570,NaN
...,...,...
32078,773640,"Nikita ""Ghost_RUS"""
32079,733530,Sacada
32080,610660,Laush Dmitriy Sergeevich
32081,658870,"xropi,stev3ns"


In [56]:
reviews1=reviews[['item_id', 'year', 'sent_an', 'binary_recommend']]

4.2 Agrupamos el dataframe reviews1 totalizando las columnas de sent?an y binary_recommend

In [57]:
reviews2 = reviews1.groupby(['item_id', 'year'], as_index=False)['sent_an', 'binary_recommend'].sum()
reviews2

C:\Users\Owner\AppData\Local\Temp\ipykernel_2944\15246976.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  reviews2 = reviews1.groupby(['item_id', 'year'], as_index=False)['sent_an', 'binary_recommend'].sum()


,item_id,year,sent_an,binary_recommend
0,10,2011,2,1
1,10,2012,6,3
2,10,2013,9,8
3,10,2014,40,27
4,10,2015,31,16
...,...,...,...,...
6759,521430,2014,2,1
6760,521570,2013,0,1
6761,521570,2015,0,1
6762,521990,2013,2,1


4.3. Unimos toda la información en un solo data frame usando la columna id

In [58]:
worstdev = pd.merge(reviews2, dfoutput, left_on='item_id', right_on='id')
worstdev

,item_id,year,sent_an,binary_recommend,id,developer
0,10,2011,2,1,10,Valve
1,10,2012,6,3,10,Valve
2,10,2013,9,8,10,Valve
3,10,2014,40,27,10,Valve
4,10,2015,31,16,10,Valve
...,...,...,...,...,...,...
5906,521430,2014,2,1,521430,David Mulder
5907,521570,2013,0,1,521570,Tamationgames
5908,521570,2015,0,1,521570,Tamationgames
5909,521990,2013,2,1,521990,Dawid Bujak


4.4. Agrupamos la información por año y por la menor calificación combinada de recomendación y sentiment analysis

In [59]:
worstdev1 = worstdev.groupby('year').apply(lambda x: x.nsmallest(3, ['binary_recommend', 'sent_an'])).reset_index(drop=True)

In [60]:
worstdev1

,item_id,year,sent_an,binary_recommend,id,developer
0,275850,2010,2,0,275850,Hello Games
1,10500,2010,0,1,10500,"The Creative Assembly,Feral Interactive (Mac),..."
2,17410,2010,0,1,17410,DICE
3,18700,2011,0,0,18700,Broken Rules
4,63940,2011,0,0,63940,1C Company
5,218620,2011,0,0,218620,NaN
6,42920,2012,0,0,42920,NeoCoreGames
7,208650,2012,0,0,208650,Rocksteady Studios
8,209100,2012,0,0,209100,Slant Six Games
9,1670,2013,0,0,1670,Strategy First


In [61]:
worstdev1.to_parquet('worstdev1.parquet')

In [62]:
worstdev1=pd.read_parquet('worstdev1.parquet')

4.5. Creamos la funcion solicitada


In [63]:
def UsersWorstDeveloper(year):
    if year not in worstdev1['year'].unique():
        return {"No se tiene información respecto al año ingresado"}
    # Filter the DataFrame based on the given year
    selected_year = worstdev1[worstdev1['year'] == year]

    # Sort the values based on the 'binary_recommend' column in descending order
    sorted_items = selected_year.sort_values(by='binary_recommend', ascending=False)

    # Create a list of dictionaries with the desired format
    result_list = [{"Puesto {}: ".format(i+1): developer} for i, developer in enumerate(sorted_items['developer'])]

    return result_list


In [64]:
UsersWorstDeveloper(2023)

{'No se tiene información respecto al año ingresado'}

## 5. FUNCIÓN def sentiment_analysis
 def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

Utilizaremos los data frames output y reviews

In [65]:
output.head()

,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
0,0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140,Kotoshiro,2018
1,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980,Secret Level SRL,2018
2,2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,0.0,670290,Poolians.com,2017
3,3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,0.0,767400,彼岸领域,2017
4,4,NaN,"['Action', 'Casual', 'Indie', 'Sports']",Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,0.0,773570,NaN,2014


In [66]:
reviews.head()

,Unnamed: 0,posted,item_id,helpful,recommend,review,user_id,user_url,year,language,sent_an,binary_recommend
0,0,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2,1
1,1,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2,1
2,2,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,en,2,1
3,3,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,2014,en,2,1
4,4,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,2013,en,2,1


5.1 Creamos un data frame con las columnas necesarias para resolver lo solicitado tanto para reviews como para output

In [67]:
reviews1=reviews[['item_id', 'year', 'sent_an']]
reviews1

,item_id,year,sent_an
0,1250,2011,2
1,22200,2011,2
2,43110,2011,2
3,251610,2014,2
4,227300,2013,2
...,...,...,...
58426,70,2013,2
58427,362890,2014,2
58428,273110,2015,2
58429,730,2015,2


In [68]:
dfoutput=output[['id','developer']]
dfoutput

,id,developer
0,761140,Kotoshiro
1,643980,Secret Level SRL
2,670290,Poolians.com
3,767400,彼岸领域
4,773570,NaN
...,...,...
32078,773640,"Nikita ""Ghost_RUS"""
32079,733530,Sacada
32080,610660,Laush Dmitriy Sergeevich
32081,658870,"xropi,stev3ns"


5.2. Unimos los dataframes en uno solo

In [69]:
sentiment=pd.merge(dfoutput,reviews1,left_on='id',right_on='item_id')

In [70]:
sentiment

,id,developer,item_id,year,sent_an
0,282010,Stainless Games Ltd,282010,2013,1
1,70,Valve,70,2015,2
2,70,Valve,70,2011,0
3,70,Valve,70,2014,2
4,70,Valve,70,2013,2
...,...,...,...,...,...
53191,80,Valve,80,2014,1
53192,80,Valve,80,2013,2
53193,80,Valve,80,2015,1
53194,80,Valve,80,2014,1


5.3. Separamos la columna sent_an (sentiment analysis) en columnas Positive, Neutral y Negative

In [71]:
sentiment['Positive'] = (sentiment['sent_an'] == 2).astype(int)
sentiment['Neutral'] = (sentiment['sent_an'] == 1).astype(int)
sentiment['Negative'] = (sentiment['sent_an'] == 0).astype(int)
sentiment

,id,developer,item_id,year,sent_an,Positive,Neutral,Negative
0,282010,Stainless Games Ltd,282010,2013,1,0,1,0
1,70,Valve,70,2015,2,1,0,0
2,70,Valve,70,2011,0,0,0,1
3,70,Valve,70,2014,2,1,0,0
4,70,Valve,70,2013,2,1,0,0
...,...,...,...,...,...,...,...,...
53191,80,Valve,80,2014,1,0,1,0
53192,80,Valve,80,2013,2,1,0,0
53193,80,Valve,80,2015,1,0,1,0
53194,80,Valve,80,2014,1,0,1,0


In [72]:
sentiment.drop(columns=['id','item_id','year','sent_an']) # eliminamos columnas innecesarias

,developer,Positive,Neutral,Negative
0,Stainless Games Ltd,0,1,0
1,Valve,1,0,0
2,Valve,0,0,1
3,Valve,1,0,0
4,Valve,1,0,0
...,...,...,...,...
53191,Valve,0,1,0
53192,Valve,1,0,0
53193,Valve,0,1,0
53194,Valve,0,1,0


5.4. Agrupamos por developer y sumamos sus calificaciones

In [73]:
sentiment1 = sentiment.groupby(['developer'], as_index=False)['Positive', 'Neutral','Negative'].sum()

C:\Users\Owner\AppData\Local\Temp\ipykernel_2944\1019479319.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sentiment1 = sentiment.groupby(['developer'], as_index=False)['Positive', 'Neutral','Negative'].sum()


In [74]:
sentiment1

,developer,Positive,Neutral,Negative
0,07th Expansion,0,2,1
1,"10th Art Studio,Adventure Productions",1,0,1
2,10tons Ltd,1,0,0
3,11 bit studios,25,9,28
4,14° East,1,0,1
...,...,...,...,...
2115,xXarabongXx,0,0,1
2116,△○□× (Miwashiba),5,0,0
2117,"インレ,Inre",3,1,1
2118,橘子班,1,2,1


In [75]:
sentiment1.to_parquet('sentimentfinal.parquet')

In [76]:
sentiment1=pd.read_parquet('sentimentfinal.parquet')

5.5. Creamos la función

In [77]:
def sentiment_analysis(developer):
    if developer not in sentiment1['developer'].unique():
        return {"No se tiene información respecto a {}" .format(developer)}
    # Filter the DataFrame for the specified developer
    developer_data = sentiment1[sentiment1['developer'] == developer]

    # Extract the total counts for each sentiment category
    negative_count = developer_data['Negative'].values[0]
    neutral_count = developer_data['Neutral'].values[0]
    positive_count = developer_data['Positive'].values[0]

    # Create a dictionary with the desired format
    result_dict = {developer: [f"Negative = {negative_count}", f"Neutral = {neutral_count}", f"Positive = {positive_count}"]}

    return result_dict

In [78]:
sentiment_analysis('xXarabongXx')

{'xXarabongXx': ['Negative = 1', 'Neutral = 0', 'Positive = 0']}